In [0]:
-- Select all columns from the traffic_incidents table
SELECT * FROM gshen_catalog.city_of_calgary.traffic_incidents;

-- Leverage AI Functions to perform simple ML classification on traffic incidents
SELECT
  *,
  ai_classify(
    description,
    ARRAY(
      "Single Vehicle Incident",
      "Two Vehicle Incident",
      "Multi-Vehicle Incident",
      "Pedestrian Involved",
      "Cyclist Involved",
      "Stalled Vehicle",
      "Hazardous Road Condition",
      "Traffic Signal Issue",
      "Train Crossing / LRT Issue",
      "Police / Emergency Response",
      "Structure Fire / Environmental Hazard",
      "Planned Closure / Maintenance",
      "Funeral Procession / Event"
    )
  ) as incident_type
FROM
  gshen_catalog.city_of_calgary.traffic_incidents
LIMIT 100;

-- Use AI_Query to extend the functionality further by assessing the impact on traffic
SELECT
  *,
  ai_classify(
    description,
    ARRAY(
      "Single Vehicle Incident",
      "Two Vehicle Incident",
      "Multi-Vehicle Incident",
      "Pedestrian Involved",
      "Cyclist Involved",
      "Stalled Vehicle",
      "Hazardous Road Condition",
      "Traffic Signal Issue",
      "Train Crossing / LRT Issue",
      "Police / Emergency Response",
      "Structure Fire / Environmental Hazard",
      "Planned Closure / Maintenance",
      "Funeral Procession / Event"
    )
  ) as incident_type,
  ai_query(
    "databricks-claude-3-7-sonnet",
    "Given the location and description of a traffic incident in Calgary Alberta. "
    | "Take into consideration how busy the road is, and the type of traffic incident."
    | "The output should be a string of one of the following: No Impact, Minor Impact, Moderate Impact, Major Impact"
    | "Do not return any other text besides: No Impact, Minor Impact, Moderate Impact, Major Impact"
    | incident_info
    | description
  ) AS impact_on_traffic
FROM
  gshen_catalog.city_of_calgary.traffic_incidents
WHERE start_dt >= current_timestamp() - INTERVAL 1 DAY;

-- Create a materialized view containing the AI Functions for traffic incidents in the last 24 hours
CREATE MATERIALIZED VIEW IF NOT EXISTS gshen_catalog.city_of_calgary.traffic_incidents_categorized_24h AS
SELECT
  *,
  ai_classify(
    description,
    ARRAY(
      "Single Vehicle Incident",
      "Two Vehicle Incident",
      "Multi-Vehicle Incident",
      "Pedestrian Involved",
      "Cyclist Involved",
      "Stalled Vehicle",
      "Hazardous Road Condition",
      "Traffic Signal Issue",
      "Train Crossing / LRT Issue",
      "Police / Emergency Response",
      "Structure Fire / Environmental Hazard",
      "Planned Closure / Maintenance",
      "Funeral Procession / Event"
    )
  ) AS incident_type,
  ai_query(
    "databricks-claude-3-7-sonnet",
    "Given the location and description of a traffic incident in Calgary Alberta. "
    | "Take into consideration how busy the road is, and the type of traffic incident. "
    | "The output should be a string of one of the following: No Impact, Minor Impact, Moderate Impact, Major Impact. "
    | "Do not return any other text besides: No Impact, Minor Impact, Moderate Impact, Major Impact. "
    | incident_info
    | description
  ) AS impact_on_traffic
FROM gshen_catalog.city_of_calgary.traffic_incidents
WHERE start_dt >= current_timestamp() - INTERVAL 1 DAY;